# Week 9: SQL for Spatial Data Processing

This tutorial covers key SQL elements for spatial data processing, focusing on:

1. Date and Time Functions
2. Type Casting and Conversion
3. Spatial Functions (PostGIS)
4. Aggregation and Grouping
5. Creating New Tables from Queries

We'll explore these concepts using examples from our lecture.

## 1. Date and Time Functions

PostgreSQL provides powerful functions for working with date and time data. Let's explore some key functions:

### 1.1 DATE_TRUNC

`DATE_TRUNC` truncates a timestamp to a specified precision (year, month, day, hour, minute, etc.).

```sql
SELECT date_trunc('minute', TIMESTAMP '2009-07-18 04:30:00-04');
```

This returns: `2009-07-18 04:30:00-04`

If we had used `'hour'` instead of `'minute'`, it would return: `2009-07-18 04:00:00-04`

### 1.2 DATE_PART

`DATE_PART` extracts a specific component (year, month, day, hour, minute, etc.) from a timestamp.

```sql
SELECT DATE_PART('minute', TIMESTAMP '2009-07-18 04:31:00-04');
```

This returns: `31` (the minute component of the timestamp)

## 2. Type Casting and Conversion

PostgreSQL allows converting between different data types using the `CAST` function or the `::` operator.

### 2.1 Basic Casting

```sql
-- Cast DATE_PART result to integer
SELECT CAST(DATE_PART('minute', TIMESTAMP '2009-07-18 04:31:00-04') AS integer);
```

This returns: `31` (as an integer, not a numeric type)

### 2.2 String Concatenation and Interval Casting

The `||` operator concatenates strings, and we can cast strings to intervals:

```sql
SELECT CAST(3 || ' minutes' AS interval);
```

This returns: `00:03:00` (a 3-minute interval)

## 3. Mathematical Functions

PostgreSQL provides various mathematical functions for data processing.

### 3.1 Modulo Operation (MOD)

The `MOD` function returns the remainder of a division operation:

```sql
SELECT mod(CAST(DATE_PART('minute', TIMESTAMP '2009-07-18 04:30:00-04') AS integer), 15);
```

This returns: `0` (because 30 divided by 15 has a remainder of 0)

If we used `04:31:00` instead, it would return: `1` (because 31 divided by 15 has a remainder of 1)

## 4. Spatial Functions (PostGIS)

PostGIS extends PostgreSQL with spatial functions for working with geographic data.

### 4.1 ST_MakeLine

`ST_MakeLine` creates a LineString from Point geometries:

```sql
SELECT ST_MakeLine(geom ORDER BY time) AS geom
FROM ch11.aussie_track_points
GROUP BY track_period;
```

This creates a line by connecting points in order of their timestamp.

### 4.2 ST_Union

`ST_Union` combines multiple geometries into a single geometry:

```sql
-- Aggregate city geometries
CREATE TABLE ch11.boroughs_city AS
SELECT ST_Union(geom) AS geom
FROM ch11.boroughs;
```

The `ST_Union` function can also take a tolerance parameter for simplification:

```sql
-- New York City unified with 500-foot grid
CREATE TABLE ch11.boroughs_citi_500 AS 
SELECT ST_Union(geom, 500) AS geom
FROM ch11.boroughs;
```

### 4.3 ST_NPoints and ST_NumGeometries

`ST_NPoints` counts the number of points in a geometry:

```sql
SELECT ST_NPoints(geom) AS np
FROM ch11.aussie_run;
```

`ST_NumGeometries` counts the number of geometries in a multi-geometry:

```sql
SELECT ST_NumGeometries(ST_Union(geom)) AS num_poly_after
FROM ch11.cities
GROUP BY city;
```

### 4.4 ST_Length

`ST_Length` calculates the length of a LineString. When used with `::geography` casting, it returns the length in meters:

```sql
SELECT CAST(ST_Length(geom::geography) AS integer) AS dist_m
FROM ch11.aussie_run;
```

### 4.5 ST_Touches

`ST_Touches` checks if two geometries share a boundary but don't overlap:

```sql
SELECT a.city, a.gid AS gid_1, b.gid AS gid_2
FROM ch11.cities a
JOIN ch11.cities b 
ON a.city = b.city  -- Same city name
AND a.gid < b.gid  -- Avoid self-join and duplicate pairs
AND ST_Touches(a.geom, b.geom);  -- Check if they share a boundary
```

## 5. Aggregation and Grouping

SQL provides powerful aggregation functions and grouping capabilities.

### 5.1 Basic Aggregation Functions

```sql
SELECT 
    MIN(time) AS t_start, 
    MAX(time) AS t_end
FROM ch11.aussie_track_points
GROUP BY track_period;
```

This finds the earliest and latest timestamps for each track period.

### 5.2 Complex Grouping with Calculated Fields

We can group by calculated fields, not just simple columns:

```sql
SELECT 
    DATE_TRUNC('minute', time) - 
        CAST(
            mod(CAST(DATE_PART('minute', time) AS integer), 15) || ' minutes' AS interval
        ) AS track_period,
    MIN(time) AS t_start, 
    MAX(time) AS t_end,
    ST_MakeLine(geom ORDER BY time) AS geom
FROM ch11.aussie_track_points
GROUP BY track_period;
```

This groups track points into 15-minute intervals by:
1. Truncating the timestamp to the minute
2. Calculating the remainder when dividing the minute by 15
3. Subtracting that remainder to get the start of the 15-minute interval

### 5.3 HAVING Clause

The `HAVING` clause filters groups after aggregation:

```sql
SELECT city, COUNT(city) AS num_records,
SUM(ST_NumGeometries(geom)) AS numpoly_before,
ST_NumGeometries(ST_Union(geom)) AS num_poly_after
FROM ch11.cities
GROUP BY city
HAVING COUNT(city) > 1;
```

This finds cities with multiple records and shows the before/after polygon counts when merged.

## 6. Creating New Tables from Queries

PostgreSQL allows creating new tables from query results using the `INTO` clause or `CREATE TABLE AS` statement.

### 6.1 Using INTO Clause

```sql
SELECT 
    DATE_TRUNC('minute',time) -
        CAST(
            mod(
                CAST(DATE_PART('minute',time) AS integer),15
            ) ||' minutes' AS interval
        ) AS track_period,
    MIN(time) AS t_start, 
    MAX(time) AS t_end, 
    ST_MakeLine(geom ORDER BY time) AS geom  
INTO ch11.aussie_run
FROM ch11.aussie_track_points
GROUP BY track_period;
```

This creates a new table `ch11.aussie_run` with the results of the query.

### 6.2 Using CREATE TABLE AS

```sql
CREATE TABLE ch11.cities_merged AS
SELECT 
    city,
    COUNT(city) AS num_records,
    SUM(ST_NumGeometries(geom)) AS numpoly_before,
    ST_NumGeometries(ST_Union(geom)) AS num_poly_after,
    ST_Union(geom) AS merged_geom
FROM ch11.cities
GROUP BY city
HAVING COUNT(city) > 0;
```

This creates a new table `ch11.cities_merged` with the results of the query.

## 7. Practical Example: Tracking Data Analysis

Let's put everything together with a practical example from our lecture. This example processes GPS tracking data to create line segments for 15-minute intervals and analyze the results.

### 7.1 Creating Track Segments

```sql
-- Create a new table with track segments
SELECT 
    DATE_TRUNC('minute',time) -
        CAST(
            mod(
                CAST(DATE_PART('minute',time) AS integer),15
            ) ||' minutes' AS interval
        ) AS track_period,
    MIN(time) AS t_start, 
    MAX(time) AS t_end, 
    ST_MakeLine(geom ORDER BY time) AS geom  
INTO ch11.aussie_run
FROM ch11.aussie_track_points
GROUP BY track_period;
```

This query:
1. Groups track points into 15-minute intervals
2. Finds the start and end times for each interval
3. Creates a line geometry connecting the points in time order
4. Saves the results to a new table

### 7.2 Analyzing the Results

```sql
SELECT 
    CAST(track_period AS timestamp),
    CAST(t_start AS timestamp) AS t_start,    
    CAST(t_end AS timestamp) AS t_end, 
    ST_NPoints(geom) AS np, 
    CAST(ST_Length(geom::geography) AS integer) AS dist_m, 
    (t_end - t_start) AS dur               
FROM ch11.aussie_run;
```

This query analyzes the track segments by:
1. Converting timestamps to a readable format
2. Counting the number of points in each line segment
3. Calculating the distance traveled in meters
4. Computing the duration of each segment

## Summary

In this tutorial, we've covered key SQL elements for spatial data processing:

1. **Date and Time Functions**: `DATE_TRUNC`, `DATE_PART`
2. **Type Casting**: `CAST`, string concatenation with `||`
3. **Mathematical Functions**: `MOD`
4. **Spatial Functions**: `ST_MakeLine`, `ST_Union`, `ST_NPoints`, `ST_NumGeometries`, `ST_Length`, `ST_Touches`
5. **Aggregation**: `MIN`, `MAX`, `COUNT`, `SUM`, `GROUP BY`, `HAVING`
6. **Table Creation**: `INTO`, `CREATE TABLE AS`

These elements are powerful tools for processing and analyzing spatial data in PostgreSQL with PostGIS.